# 객체

::: {.hidden}

## 참고 문헌

- [Pro Git, Git Internels, Git Objects](chrome-extension://oemmndcbldboiebfnladdacbdfmadadm/file:///Users/joelkim/Work/study/study_cs/book/DevOps/Progit/progit.pdf#%5B%7B%22num%22%3A2221%2C%22gen%22%3A0%7D%2C%7B%22name%22%3A%22XYZ%22%7D%2C0%2C510.57%2Cnull%5D)
- [Mastering Git, Ch10 Keeping History Clean, An introduction to Git internals](chrome-extension://oemmndcbldboiebfnladdacbdfmadadm/file:///Users/joelkim/Work/study/study_cs/book/DevOps/Narebski/2024%20-%20Narebski%20-%20Mastering%20Git,%202nd%20Edition.pdf#p263)

:::

## 객체

- git의 모든 정보는 객체(object) 형태로 저장
- `commit` 명령으로 레포지토리에 정식 저장된 정보 뿐 아니라 `add` 명령으로 스테이지에 저장된 파일도 객체로 저장됨

## 해시값

- 모든 객체는 이름(아이디)로 sha1 해시값 40글자를 가짐
- 해시값은 객체의 내부정보 뿐 아니라 작성자 정보, 작성 시간 정보 등을 통합하여 계산하므로 동일한 해시가 발생할 가능성이 매우 적음

```bash
echo 'test content' | git hash-object -w --stdin
```

## 객체 저장 위치

- 객체는 `.git/objects` 디렉토리 아래에 서브디렉토리로 나뉘어서 파일형태로 저장
- 40글자 sha1 해시값 중 첫 2글자를 디렉토리 이름으로 하고 나머지 38글자를 파일이름으로 저장
- 추가 디렉토리
  - `.git/objects/info`
  - `.git/objects/pack`

```bash
find .git/objects -type f
```

## 객체의 종류

- blob 객체: 한 순간의 파일 하나의 스냅샷 정보
- tree 객체: 한 순간의 디렉토리 트리 구조 정보
- commit 객체: 하나의 커밋 정보

## 객체를 보는 명령

- 저장소 전체의 객체 목록을 보는 법

```bash
git cat-file --batch-check --batch-all-objects --unordered
```

- 현재 커밋을 구성하는 객체 목록을 보는 법

```bash
git rev-list --objects --all
```

- 객체 하나의 유형을 보는 법

```bash
git cat-file -t 해시값
```

- 객체 하나의 정보를 보는 법

```bash
git cat-file -p 해시값
```

## blob 객체

## tree 객체

## commit 객체